In [1]:
import pandas as pd
import os
import itertools
import numpy as np

# Own functions 
from emelab_tools import *

# DF idalumno

data_tot_mrun >>> df con 'agno','grado','idalumno','mrun','rbd' para unir ID estudiante ACE con MINEDUC

In [3]:
# GET MRUN 
data_tot_mrun = pd.DataFrame(columns=['agno','grado','idalumno','mrun','rbd'])

no_pje_order = ['Con puntaje','Inasistencia','Prueba en blanco', 'Estudiante Integrado o no habla espaÃ±ol', 
                'Estudiante Integrado o no habla espaï¿½ol','No rinde la prueba','Prueba nula','Perdida de material','Estudiante retirado']

cohort_number = 1
for cohort in [T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11_pandemia,T12,T13,T14,T15,CS1,CS2,CS3,CS4, CS5_pandemia]:
    # walk cohorts
    for t in cohort:
        curso =  t[5:7]
        agno = t[:4]
        # walk subsolders
        for root, dirs, files in os.walk(path):
            # walk all files in subfolders
             for file in files:
                if curso+agno+'_alu' in file and (('csv' in file) | ('txt' in file)):
                    path_file = os.path.join(root, file)
                    print(file)
                    data_mrun = pd.read_csv(path_file,encoding = 'unicode_escape', sep="|", header='infer', low_memory=False)
                    data_mrun.columns = [x.lower() for x in data_mrun.columns]
                    # Botar duplicados idalumno quedandonos con los que tienen ptje 
                    no_ptje = 'noptje_lect' + curso + '_alu' 
                    if (no_ptje in data_mrun.columns) | ('marca_lect'+curso+'_alu' in data_mrun.columns):
                        data_mrun = data_mrun.rename(columns = {'marca_lect'+curso+'_alu':no_ptje})
                        if '2m2013' in file:
                            data_mrun = data_mrun.sort_values(by = ['mrun', 'ptje_lect2m_alu','ptje_mate2m_alu'], ascending=False)
                        else:
                            data_mrun[no_ptje] = data_mrun[no_ptje].astype('category')
                            new_vals = data_mrun[no_ptje].unique()
                            no_pje_order_final = [x for x in no_pje_order if x in new_vals]
                            no_pje_order_final = no_pje_order_final + [x for x in new_vals if x not in no_pje_order_final]
                            no_pje_order_final = [x for x in no_pje_order_final if str(x)!= 'nan']
                            data_mrun[no_ptje] = data_mrun[no_ptje].cat.reorder_categories(no_pje_order_final)
                            # duplicados con puntaje quedan primero 
                            data_mrun.sort_values(by = ['mrun', no_ptje], ascending=True, inplace=True)
                            
                    elif '3m2012' in file:
                            data_mrun.sort_values(by = ['mrun', 'ptje_rea3m_alu','ptje_lis3m_alu'], ascending=False, inplace=True)
                            print(data_mrun.grado.unique())
                    else:
                        cols_ordenar = ['ptje_lectX_alu', 'noptje_aus_lectX_alu', 'noptje_bco_lectX_alu', 'noptje_neepX_alu',
                                        'noptje_nespX_alu',  'noptje_nul_lectX_alu', 'noptje_ppm_mateX_alu', 'noptje_retX_alu']
                        cols_ordenar = [x.replace('X', curso) for x in cols_ordenar]
                        data_mrun.sort_values(by = ['mrun'] + cols_ordenar, 
                                              ascending= [True, True, False,False,False,False,False,False,False], inplace=True)
                    print(len(data_mrun))
                    #data_mrun.drop_duplicates(subset= ['mrun'], keep = 'first', inplace=True)
                    data_mrun['is_duplicate'] = data_mrun.duplicated(subset=['mrun'], keep='first')

                    print(len(data_mrun.mrun.unique()))
                    data_mrun = data_mrun[['agno','grado','idalumno','mrun','rbd', 'is_duplicate']]
                    data_mrun['cohort_number'] = cohort_number
                    data_tot_mrun = pd.concat([data_tot_mrun,data_mrun])
                    
    cohort_number += 1 
    
print(data_tot_mrun.cohort_number.value_counts())
data_tot_mrun.sort_values(by='agno', inplace=True)

simce4b2005_alu_mrun_privada_final-SEG.csv
277804
269367
simce8b2009_alu_mrun_privadas_final-SEG.csv
266269
260634
simce3m2012_alu_mrun_publica_final-SEG.csv
['3m']
223507
220218
simce4b2007_alu_mrun_privada_final-SEG.csv
267451
258425
simce8b2011_alu_mrun_privada_final-SEG.csv
250473
246176
simce2m2013_alu_mrun_privada_final-SEG.csv
254580
241768
simce4b2008_alu_mrun_privada_final-SEG.csv
264120
255712
simce2m2014_alu_mrun_privada_final-SEG.csv
241730
233865
simce4b2009_alu_mrun_privadas_final-SEG.csv
254823
247743
simce8b2013_alu_mrun_privada_final-SEG.csv
255132
245920
simce2m2015_alu_mrun_privadas_final-SEG.csv
245160
233278
simce4b2010_alu_mrun_privadas_final-SEG.csv
255448
251839
simce8b2014_alu_mrun_privada_final-SEG.csv
256983
248951
simce2m2016_alu_mrun_privada_final_SEG.csv
248158
237797
simce4b2011_alu_mrun_privada_final-SEG.csv
247666
242489
simce6b2013_alu_mrun_privada_final-SEG.csv
262421
250138
simce8b2015_alu_mrun_privadas_final-SEG.csv
255607
245440
simce2m2017_alu_mru

## Eliminar duplicados agno - mrun

In [4]:
data_tot_mrun.is_duplicate.value_counts()

is_duplicate
False    11560388
True       484566
Name: count, dtype: int64

In [5]:
pd.crosstab(data_tot_mrun.is_duplicate,data_tot_mrun.agno)

agno,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2022
is_duplicate,,,,,,,,,,,,,,,,,,,
False,282948,252399,293329,269367,537862,534337,520685,508377,505640,488665,931221,1206320,1201141,1200797,716504,716117,737813,239393,417473
True,6812,7181,5676,8437,17239,15200,14332,12715,7980,9474,22542,60326,45280,57847,28360,32855,36181,10104,86025


In [6]:
pd.crosstab(data_tot_mrun.is_duplicate,data_tot_mrun.agno, normalize = 'columns')

agno,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2022
is_duplicate,,,,,,,,,,,,,,,,,,,
False,0.976491,0.972336,0.981017,0.96963,0.968944,0.97234,0.973212,0.975599,0.984463,0.980981,0.976365,0.952373,0.963672,0.95404,0.961926,0.956133,0.953254,0.959503,0.829145
True,0.023509,0.027664,0.018983,0.03037,0.031056,0.02766,0.026788,0.024401,0.015537,0.019019,0.023635,0.047627,0.036328,0.04596,0.038074,0.043867,0.046746,0.040497,0.170855


In [132]:
data_tot_mrun = data_tot_mrun[data_tot_mrun.is_duplicate==False]

## Elminar trayectorias inconsistentes

In [8]:
# generar curso numerico
data_tot_mrun['grado_num'] = np.nan
data_tot_mrun.loc[data_tot_mrun.grado == '2b', 'grado_num'] = 2
data_tot_mrun.loc[data_tot_mrun.grado == '4b', 'grado_num'] = 4
data_tot_mrun.loc[data_tot_mrun.grado == '6b', 'grado_num'] = 6
data_tot_mrun.loc[data_tot_mrun.grado == '8b', 'grado_num'] = 8
data_tot_mrun.loc[data_tot_mrun.grado == '2m', 'grado_num'] = 10
data_tot_mrun.loc[data_tot_mrun.grado == '3m', 'grado_num'] = 11
# generar col
data_tot_mrun['agno_menos_grado'] = data_tot_mrun.agno - data_tot_mrun.grado_num
# identificar trayectorias consistentes 
consistent_trajectories = data_tot_mrun.groupby('mrun', as_index=False)['grado_num'].apply(lambda x: x.is_monotonic_increasing)

In [9]:
# N inconsistente
consistent_trajectories.grado_num.value_counts()

grado_num
True     5176674
False       4183
Name: count, dtype: int64

In [10]:
# generar todas las combinaciones posibles de años y cursos 
def findsubsets(s, n):
    return list(itertools.combinations(s, n))

path_grade = [2,2,2,2,2,4,4,4,4,4,6,6,6,6,6,8,8,8,8,8,10,10,10,10,10,11,11,11,11,11]
path_year = list(range(2002,2020)) + [2022]
max_path_grade = []
max_path_year = []
for n in [2,3,4,5]:
    print(n)
    max_path_grade += findsubsets(path_grade, n)
    max_path_year += findsubsets(path_year, n)

2
3
4
5


In [11]:
for col in ['agno', 'idalumno', 'cohort_number','grado_num', 'agno_menos_grado']:
    print(col)
    data_tot_mrun[col] = data_tot_mrun[col].astype(int)

for col in ['mrun', 'rbd']:
    print(col)
    data_tot_mrun[col] = data_tot_mrun[col].astype(float)

agno
idalumno
cohort_number
grado_num
agno_menos_grado
mrun
rbd


In [12]:
# trayectorias no consistentes: pasar por 5 pruebas para eliminar filas malas
non_consistent = list(consistent_trajectories[consistent_trajectories.grado_num == False].mrun)
df_consistent = pd.DataFrame(columns = data_tot_mrun.columns)
c = 0
N = len(non_consistent)
len_1 = []
for s in non_consistent:
    # counting
    if c%100 == 0:
        print(np.round(100*c/N),'%')
    c+=1
    
    df = data_tot_mrun[data_tot_mrun.mrun == s]
    # if more than two points, try to get a subset that is correct
    if len(df) > 2:
        ind = df.index
        dict_df = {x : 0 for x in ind}
        
        for i in ind:
            df_i = df[df.index != i]

            if (tuple(df_i.grado_num.values) in max_path_grade): 
                dict_df[i] +=1

            if (tuple(df_i.agno.values) in max_path_year): # & ():
                dict_df[i] +=1

            if df_i.cohort_number.is_monotonic_increasing:
                dict_df[i] +=1

            if df_i.agno_menos_grado.is_monotonic_increasing:
                dict_df[i] +=1
        
        m = max(dict_df.values())
        i_out = [i for i in dict_df if dict_df[i]==m][0]
        df_consistent = pd.concat([df_consistent,df[df.index != i_out]])
        
    else:
        len_1+=[s]
        
# De aquellas trayectorias inconsistentes de largo 2, dejar una observación al azar
df_inconsitent_short = data_tot_mrun[data_tot_mrun.mrun.isin(len_1)]
df_inconsitent_short = df_inconsitent_short.sample(frac=1)
df_inconsitent_short = df_inconsitent_short.drop_duplicates(subset=['mrun'],keep='first')
# concat
df_consistent = pd.concat([df_consistent,df_inconsitent_short])

0.0 %
2.0 %
5.0 %
7.0 %
10.0 %
12.0 %
14.0 %
17.0 %
19.0 %
22.0 %
24.0 %
26.0 %
29.0 %
31.0 %
33.0 %
36.0 %
38.0 %
41.0 %
43.0 %
45.0 %
48.0 %
50.0 %
53.0 %
55.0 %
57.0 %
60.0 %
62.0 %
65.0 %
67.0 %
69.0 %
72.0 %
74.0 %
77.0 %
79.0 %
81.0 %
84.0 %
86.0 %
88.0 %
91.0 %
93.0 %
96.0 %
98.0 %


In [13]:
for col in ['agno', 'idalumno', 'cohort_number','grado_num', 'agno_menos_grado']:
    print(col)
    df_consistent[col] = df_consistent[col].astype(int)

for col in ['mrun', 'rbd']:
    print(col)
    df_consistent[col] = df_consistent[col].astype(float)

agno
idalumno
cohort_number
grado_num
agno_menos_grado
mrun
rbd


In [14]:
# sacar todo lo inconsistente
data_tot_mrun2 = data_tot_mrun[~data_tot_mrun.mrun.isin(non_consistent)]
# obtener nuevas trayectorias consistentes (sacando las de largo 2 y las que aún eran incosistentes)
df = df_consistent.groupby('mrun', as_index=False)['grado_num'].apply(lambda x: x.is_monotonic_increasing)
mrun_consistentes = df[df.grado_num == True].mrun.unique()
# nuevas filas a agregar
df_consistent2 = df_consistent[df_consistent.mrun.isin(mrun_consistentes)]
data_tot_mrun2 = pd.concat([data_tot_mrun2,df_consistent2])

Quedan 1,400 filas con repetidos [mrun, agno], pues aún no podemos saber cual es el correcto. Al cruzar con matrícula oficial hay que subsanar esto.

## Guardar 

In [15]:
# Guardar 
data_tot_mrun2.drop(columns= ['grado_num', 'cohort_number', 'agno_menos_grado'],inplace=True)
data_tot_mrun2.rename(columns={'grado':'curso'},inplace=True)
data_tot_mrun2['in_simce'] = 1 
data_tot_mrun2.to_csv('Datos\Intermedios\Agencia\20231017_idalumno_match_mrun_2002_2022.csv', index=False) # mrun_for_idalumnoT15_cs

# DF enrollment grade 1

- MAT_1b_data_acum >> 'agno', 'mrun', 'n_cohorte'
- Estudio de casos fill nan 

In [213]:
# OBTENER TODO 1B PARA 11 PRIMERAS TRAYECTORIAS
MAT_1b_data_acum = pd.DataFrame(columns = ['agno', 'mrun'])

# 2002 y 2003 from Rendimiento 
for file in [r'\REND_' + str(x) + '.csv' for x in [2,3]] + [r'\MAT_' + str(x) + '.csv' for x in [4,5,6,7,8,9,10,11,12,13]]:
    print(file)
    
    try:
        MAT_data = pd.read_csv(EE_path + file,encoding = 'unicode_escape', sep = ';', low_memory=False, usecols=['AGNO','MRUN','COD_ENSE','COD_GRADO'])
    except: 
        MAT_data = pd.read_csv(EE_path + file,encoding = 'unicode_escape', sep = ';', low_memory=False, usecols=['agno','mrun','cod_ense','cod_grado'])
    
    MAT_data.columns = [x.lower() for x in MAT_data.columns]
    MAT_data.mrun.replace(" ",np.nan,inplace=True)
    MAT_data['mrun'] = pd.to_numeric(MAT_data.mrun)
    MAT_data['in_1b'] = (MAT_data.cod_ense == 110) & (MAT_data.cod_grado == 1)
    MAT_data = MAT_data[MAT_data.in_1b==1]    
    MAT_data = MAT_data.drop_duplicates(subset=['mrun'], keep = 'first')  
    print(len(MAT_data))
    
    MAT_1b_data_acum = pd.concat([MAT_1b_data_acum, MAT_data[['agno', 'mrun']]])

# Dejar sólo primera aparición en grade 1 (EXCEPTO 2002 QUE NO VEMOS REPITENTES DESDE 2001)
MAT_1b_data_acum = MAT_1b_data_acum.dropna(subset='mrun')
MAT_1b_data_acum = MAT_1b_data_acum.sort_values(by=['mrun','agno'], ascending=True)
MAT_1b_data_acum['appearance_count'] = MAT_1b_data_acum.groupby(['mrun']).cumcount() + 1
MAT_1b_data_acum = MAT_1b_data_acum[MAT_1b_data_acum.appearance_count == 1]
MAT_1b_data_acum = MAT_1b_data_acum.drop(columns = ['appearance_count'])

MAT_1b_data_acum.to_csv(r'Datos\Intermedios\DatosAbiertos\matricula_grade1_mrun_T1_T11_2002_2013.csv', index=False) # matricula_1b_T1_to_T10_cohorts_mrun

\REND_2.csv
268714
\REND_3.csv
268428
\MAT_4.csv
262578
\MAT_5.csv
260194
\MAT_6.csv
254635
\MAT_7.csv
259015
\MAT_8.csv
250981
\MAT_9.csv
250063
\MAT_10.csv
241874
\MAT_11.csv
238385
\MAT_12.csv
237651
\MAT_13.csv
242683


# DF schools
EE_data_acum >>> 'agno','rbd','cod_depe','rural_rbd'

In [21]:
EE_data_acum = pd.DataFrame(columns = ['agno','rbd','cod_depe','rural_rbd','cod_com_rbd','cod_reg_rbd'])
for file in [r'\EE_' + str(x) + '.csv' for x in range(4,20)] + [r'\EE_22.csv']:
    print(file)
    for char in  [r'\EE_' + str(x) + '.csv' for x in range(4,13)]+ [r'\EE_22.csv']:
        if file == char:
            EE_data = pd.read_csv(EE_path + file,sep = ';')

    for char in  [r'\EE_' + str(x) + '.csv' for x in range(13,20)] :
        if file == char:
            EE_data = pd.read_csv(EE_path + file)
        
    if 'ïagno' in EE_data.columns:
        EE_data.rename(columns={'ïagno':'agno'},inplace=True)

    EE_data.columns = [x.lower() for x in EE_data.columns]

    if '_4' in file:
        EE_data['cod_reg_rbd'] = np.nan
        
    if '_5' in file:
        EE_data_acum.drop(columns = ['cod_reg_rbd'], inplace=True)
        EE_data_acum = pd.merge(EE_data_acum,EE_data[['cod_com_rbd','cod_reg_rbd']].drop_duplicates(), on = 'cod_com_rbd', how = 'left' )
    
    EE_data_acum = pd.concat([EE_data_acum, EE_data[['agno','rbd','cod_depe','rural_rbd','cod_com_rbd','cod_reg_rbd']]])
    print(EE_data.cod_depe.value_counts())

# Ajustar dependencia 
# Original
#1: Corporación Municipal
#2: Municipal DAEM
#3: Particular Subvencionado
#4: Particular Pagado (o no subvencionado)
#5: Corporación de Administración Delegada (DL 3166)
#6: Servicio Local de Educación
    
# Municipal
EE_data_acum['cod_depe'] = EE_data_acum.cod_depe.replace(2,1)
# Particular subvencionado
EE_data_acum['cod_depe'] = EE_data_acum.cod_depe.replace(3,2)
EE_data_acum['cod_depe'] = EE_data_acum.cod_depe.replace(5,2)
# SLEP a municipal
EE_data_acum['cod_depe'] = EE_data_acum.cod_depe.replace(6,1)
# Privados con número 3 
EE_data_acum['cod_depe'] = EE_data_acum.cod_depe.replace(4,3)

# 2002 y 2003 se repiten por no disponibilidad 
EE_2002 = EE_data_acum[EE_data_acum.agno == 2004]
EE_2002['agno'] = 2002
EE_2003 = EE_data_acum[EE_data_acum.agno == 2004]
EE_2003['agno'] = 2003

EE_data_acum = pd.concat([EE_data_acum, EE_2002])
EE_data_acum = pd.concat([EE_data_acum, EE_2003])

EE_data_acum.to_csv(r'Datos\Intermedios\DatosAbiertos\20231117_schools_2002_2012.csv', index=False)

\EE_4.csv
cod_depe
2    5013
3    4269
1    1082
4     860
5      65
Name: count, dtype: int64
\EE_5.csv
cod_depe
2    4883
3    4630
1    1215
4     763
5      70
Name: count, dtype: int64
\EE_6.csv
cod_depe
3    4891
2    4780
1    1191
4     733
5      70
Name: count, dtype: int64
\EE_7.csv
cod_depe
3    5044
2    4719
1    1189
4     728
5      70
Name: count, dtype: int64
\EE_8.csv
cod_depe
3    5252
2    4660
1    1185
4     727
5      70
Name: count, dtype: int64
\EE_9.csv
cod_depe
3    5545
2    4634
1    1177
4     671
5      70
Name: count, dtype: int64
\EE_10.csv
cod_depe
3    5674
2    4560
1    1166
4     674
5      70
Name: count, dtype: int64
\EE_11.csv
cod_depe
3    5756
2    4439
1    1141
4     657
5      70
Name: count, dtype: int64
\EE_12.csv
cod_depe
3    5965
2    4383
1    1131
4     625
5      70
Name: count, dtype: int64
\EE_13.csv
cod_depe
3    6017
2    4309
1    1116
4     602
5      70
Name: count, dtype: int64
\EE_14.csv
cod_depe
3    6065
2    4225
1    1

C:\Users\JoaquinFarina\AppData\Local\Temp\ipykernel_23936\162591542.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EE_2002['agno'] = 2002
C:\Users\JoaquinFarina\AppData\Local\Temp\ipykernel_23936\162591542.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EE_2003['agno'] = 2003


# DF GPA + grade retention + attendance

In [2]:
def map_cod_curso(row):
    ense = row['cod_ense']
    grado = row['cod_grado']

    if ense == 110 and grado < 10:
        return [grado, np.nan]
    elif grado == 1 and ense in [310, 410, 510, 610, 710, 810, 910]:
        return [9, np.nan]
    elif grado == 2 and ense in [310, 410, 510, 610, 710, 810, 910]:
        return [10,np.nan]
    elif grado == 3 and ense == 310:
        return [11, 'HC']
    elif grado == 3 and ense in [410, 510, 610, 710, 810, 910]:
        return [11, 'TP']
    elif grado == 4 and ense == 310:
        return [12, 'HC']
    elif grado == 4 and ense in [410, 510, 610, 710, 810, 910]:
        return [12, 'TP']
    else:
        return [np.nan, np.nan]

In [7]:
# GPA 
# ojo, tom tiempo y ya estpa guardaado el csv 
REND_data_acum = pd.DataFrame(columns = ['agno','rbd', 'mrun','curso','gen_alu' , 'HS_type','prom_gral','asistencia', 'sit_fin'])

for file in [r'\REND_' + str(x) + '.csv' for x in range(2,23)]:
    print(file)
    
    if file in ['\\REND_16.csv','\\REND_6.csv','\\REND_18.csv','\\REND_19.csv','\\REND_20.csv','\\REND_21.csv','\\REND_22.csv']:
        try:
            REND_data = pd.read_csv(EE_path + file,encoding = 'unicode_escape', sep = ';', low_memory=False, usecols=['ï»¿AGNO','RBD','MRUN','COD_ENSE','COD_GRADO','GEN_ALU', 'PROM_GRAL', 'ASISTENCIA', 'SIT_FIN'])
            REND_data.rename(columns = {'ï»¿AGNO':'AGNO'},inplace=True)
        except: 
            REND_data = pd.read_csv(EE_path + file,encoding = 'unicode_escape', sep = ';', low_memory=False, usecols=['ï»¿agno', 'rbd','mrun','cod_ense','cod_grado','gen_alu' , 'prom_gral','asistencia','sit_fin'])
            REND_data.rename(columns = {'ï»¿agno':'agno'},inplace=True)
    
    else:
        try:
            REND_data = pd.read_csv(EE_path + file,encoding = 'unicode_escape', sep = ';', low_memory=False, usecols=['AGNO','RBD','MRUN','COD_ENSE','COD_GRADO','GEN_ALU', 'PROM_GRAL', 'ASISTENCIA', 'SIT_FIN'])
        except: 
            REND_data = pd.read_csv(EE_path + file,encoding = 'unicode_escape', sep = ';', low_memory=False, usecols=['agno', 'rbd','mrun','cod_ense','cod_grado','gen_alu' , 'prom_gral','asistencia','sit_fin'])
 
    REND_data.columns = [x.lower() for x in REND_data.columns]
    
    # Apply the function to create a new column 'curso_and_HS_type'
    REND_data['curso_and_HS_type'] = REND_data.apply(map_cod_curso, axis=1)

    # Split the 'curso_and_HS_type' column into two separate columns 'curso' and 'HS_type'
    REND_data[['curso', 'HS_type']] = pd.DataFrame(REND_data['curso_and_HS_type'].tolist(), index=REND_data.index)

    # Drop the intermediate 'curso_and_HS_type' column if needed
    REND_data.drop(columns=['curso_and_HS_type'], inplace=True)
    
    REND_data.sort_values(by=['mrun','sit_fin'],inplace=True)
    REND_data.drop_duplicates(subset='mrun',keep='first', inplace=True)

    REND_data_acum = pd.concat([REND_data_acum,REND_data[['agno','rbd', 'mrun','curso','gen_alu' ,'HS_type', 'prom_gral','asistencia', 'sit_fin']]])
    
    print(REND_data.info())

REND_data_acum['prom_gral'] = REND_data_acum['prom_gral'].str.replace(',','.')
REND_data_acum['prom_gral'] = pd.to_numeric(REND_data_acum['prom_gral'])

\REND_2.csv
<class 'pandas.core.frame.DataFrame'>
Index: 3293117 entries, 4185 to 1738042
Data columns (total 11 columns):
 #   Column      Dtype  
---  ------      -----  
 0   agno        int64  
 1   rbd         int64  
 2   cod_ense    int64  
 3   cod_grado   int64  
 4   mrun        object 
 5   gen_alu     int64  
 6   prom_gral   object 
 7   asistencia  int64  
 8   sit_fin     object 
 9   curso       float64
 10  HS_type     object 
dtypes: float64(1), int64(6), object(4)
memory usage: 301.5+ MB
None
\REND_3.csv
<class 'pandas.core.frame.DataFrame'>
Index: 3396097 entries, 4213 to 1767934
Data columns (total 11 columns):
 #   Column      Dtype  
---  ------      -----  
 0   agno        int64  
 1   rbd         int64  
 2   cod_ense    int64  
 3   cod_grado   int64  
 4   mrun        object 
 5   gen_alu     int64  
 6   prom_gral   object 
 7   asistencia  int64  
 8   sit_fin     object 
 9   curso       float64
 10  HS_type     object 
dtypes: float64(1), int64(6), objec

<class 'pandas.core.frame.DataFrame'>
Index: 3084969 entries, 2173139 to 1572854
Data columns (total 11 columns):
 #   Column      Dtype  
---  ------      -----  
 0   agno        int64  
 1   rbd         int64  
 2   cod_ense    int64  
 3   cod_grado   int64  
 4   mrun        int64  
 5   gen_alu     int64  
 6   prom_gral   object 
 7   asistencia  int64  
 8   sit_fin     object 
 9   curso       float64
 10  HS_type     object 
dtypes: float64(1), int64(7), object(3)
memory usage: 282.4+ MB
None
\REND_19.csv
<class 'pandas.core.frame.DataFrame'>
Index: 3110056 entries, 2193476 to 167089
Data columns (total 11 columns):
 #   Column      Dtype  
---  ------      -----  
 0   agno        int64  
 1   rbd         int64  
 2   cod_ense    int64  
 3   cod_grado   int64  
 4   mrun        int64  
 5   gen_alu     int64  
 6   prom_gral   object 
 7   asistencia  int64  
 8   sit_fin     object 
 9   curso       float64
 10  HS_type     object 
dtypes: float64(1), int64(7), object(3)
m

In [8]:
REND_data_acum = REND_data_acum.dropna(subset = 'curso')
REND_data_acum['ranking_group'] = REND_data_acum['agno'].astype(str) + REND_data_acum['rbd'].astype(str) + REND_data_acum['curso'].astype(str)
# set minimum level
REND_data_acum.loc[REND_data_acum.prom_gral<2.0, 'prom_gral'] = 2.0

In [9]:
def get_percentil_nota(df):
    print('total :', len(df))
    cols = ['agno', 'rbd', 'mrun', 'curso','HS_type', 'prom_gral', 'asistencia','sit_fin', 'percentil_nota']

    # Initialize the result DataFrame
    df_percentil_nota = pd.DataFrame(columns=cols)

    # Group the DataFrame by 'agno' and 'rbd'
    grouped = df.groupby(['agno', 'rbd', 'curso'])

    # Define a function to calculate percentiles and add the result to the DataFrame
    def calculate_percentiles(group):
        group['percentil_nota'] = group['prom_gral'].rank(pct=True)
        return group[cols]

    # Apply the function to each group and concatenate the results
    df_percentil_nota['GPA_percentil'] = df.groupby(['agno', 'rbd', 'curso']).apply(lambda x: x.rank(pct=True))

    return df_percentil_nota

In [10]:
tot_rbd = list(REND_data_acum.rbd.unique())
chunk_size = 100
tot_data = []
for i in range(0, int(len(tot_rbd)/100) +1):
    print(i)
    chunk = tot_rbd[i*100:(i+1)*100]
    
    REND_data_acum_pct_case = REND_data_acum[REND_data_acum.rbd.isin(chunk)]
    REND_data_acum_pct_case.loc[:, 'GPA_percentil'] =  REND_data_acum_pct_case.groupby("ranking_group")["prom_gral"].rank(pct = True)

    
    if i == 0:
        REND_data_acum_pct = REND_data_acum_pct_case.copy()
    else:
        REND_data_acum_pct = pd.concat([REND_data_acum_pct, REND_data_acum_pct_case])
        

0


C:\Users\JoaquinFarina\AppData\Local\Temp\ipykernel_16524\828017466.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  REND_data_acum_pct_case.loc[:, 'GPA_percentil'] =  REND_data_acum_pct_case.groupby("ranking_group")["prom_gral"].rank(pct = True)


1


C:\Users\JoaquinFarina\AppData\Local\Temp\ipykernel_16524\828017466.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  REND_data_acum_pct_case.loc[:, 'GPA_percentil'] =  REND_data_acum_pct_case.groupby("ranking_group")["prom_gral"].rank(pct = True)


2


...

C:\Users\JoaquinFarina\AppData\Local\Temp\ipykernel_16524\828017466.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  REND_data_acum_pct_case.loc[:, 'GPA_percentil'] =  REND_data_acum_pct_case.groupby("ranking_group")["prom_gral"].rank(pct = True)


109


C:\Users\JoaquinFarina\AppData\Local\Temp\ipykernel_16524\828017466.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  REND_data_acum_pct_case.loc[:, 'GPA_percentil'] =  REND_data_acum_pct_case.groupby("ranking_group")["prom_gral"].rank(pct = True)


110


C:\Users\JoaquinFarina\AppData\Local\Temp\ipykernel_16524\828017466.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  REND_data_acum_pct_case.loc[:, 'GPA_percentil'] =  REND_data_acum_pct_case.groupby("ranking_group")["prom_gral"].rank(pct = True)


In [16]:
REND_data_acum_pct.to_csv(r'Datos\Intermedios\DatosAbiertos\20231127_rendimiento_asistencia_sit_fin_2002_2023.csv', index=False) # rendimiento_asistencia_sit_fin_2004_2019_curso_rbd_mrun

# DF school vulnerability

In [47]:
dict_year_grade_type = {}
IVE_EE_10_19 = pd.DataFrame(columns = ['agno','ciclo','RBD','p1_porcentaje','IVE'])
for root, dirs, files in os.walk(EE_path):
     for file in files:
            if ('IVE' in file) & ('~' not in file):
                path_file = os.path.join(root, file)
                data = pd.ExcelFile(path_file)
                
                basica = pd.read_excel(data, 'BASICA')
                
                if 'xlsx' not in path_file:
                    a = path_file[-6:-4]
                else:
                    a = path_file[-7:-5]
                    
                basica['agno'] = 2000 + int(a)
                basica['ciclo'] = 'b'
                total_matricula_col = [x for x in basica.columns if 'TOTAL' in x][0]
                basica['p1_porcentaje'] = basica['PRIMERA PRIORIDAD']/basica[total_matricula_col]
                ive_col = [x for x in basica.columns if 'SINAE' in x][0]
                basica.rename(columns = {ive_col:'IVE'},inplace=True) 
                basica = basica[['agno','ciclo','RBD','p1_porcentaje','IVE']]
                
                
                media = pd.read_excel(data, 'MEDIA')
                year = 2000 + int(a)
                media['agno'] = year
                media['ciclo'] = 'm'
                total_matricula_col = [x for x in media.columns if 'TOTAL' in x][0]
                media['p1_porcentaje'] = media['PRIMERA PRIORIDAD']/media[total_matricula_col]
                ive_col = [x for x in media.columns if 'SINAE' in x][0]
                media.rename(columns = {ive_col:'IVE'},inplace=True) 
                media = media[['agno','ciclo','RBD','p1_porcentaje','IVE']]                
                
                print(file)
                
                if year in [2010,2011,2015]:
                    media['IVE'] = media.IVE/100
                    basica['IVE'] = basica.IVE/100
                
                IVE_EE_10_19 = pd.concat([IVE_EE_10_19,basica])
                IVE_EE_10_19 = pd.concat([IVE_EE_10_19,media])
                

IVE_EE_10_19 = IVE_EE_10_19[~IVE_EE_10_19.RBD.isin(['TOTAL GENERAL','Total general'])]
IVE_EE_10_19.rename(columns={'RBD':'rbd'},inplace=True)
IVE_EE_10_19['rbd'] = pd.to_numeric(IVE_EE_10_19.rbd)
IVE_EE_10_19.rename(columns={'agno_idalumno':'agno'},inplace=True)
IVE_EE_10_19.to_csv(r'Datos\Intermedios\DatosAbiertos\IVE_total_10_19.csv', index = False)

IVE_10.xls
IVE_11.xls
IVE_12.xlsx
IVE_13.xlsx
IVE_14.xlsx
IVE_15.xlsx


C:\Users\JoaquinFarina\anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\JoaquinFarina\anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


IVE_16.xlsx
IVE_17.xlsx
IVE_18.xlsx
IVE_19.xlsx
